<a href="https://colab.research.google.com/github/mmmovania/CUDA_Spring2023/blob/main/Week15/HelloPyCUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 68 kB 6.4 MB/s 
     |████████████████████████████████| 78 kB 6.0 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=4cf7a95923d07158b00b81f21db5152d6455b384d6fe66b9d2030b50eb6b18f0
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2022.1.5-py2.py3-none-any.whl size=64596 sha256=c60d325f2730c63617f220b8a96fda554c06c8807adf5500adc1476866b71232
  Stored in directory: /root/.cache/pip/wheels/10/a4/a2/e42c3619d996670485ad2116d212c6521d4a1e3db7f0c20891
Successfully built pycuda pytools


In [ ]:
import pycuda.autoinit
import pycuda.driver as cuda

(free,total)=cuda.mem_get_info()
print("Global memory occupancy:%f%% free"%(free*100/total))

for devicenum in range(cuda.Device.count()):
    device=cuda.Device(devicenum)
    attrs=device.get_attributes()

    #Beyond this point is just pretty printing
    print("\n===Attributes for device %d"%devicenum)
    for (key,value) in attrs.items():
        print("%s:%s"%(str(key),str(value)))

Global memory occupancy:99.446080% free

===Attributes for device 0
ASYNC_ENGINE_COUNT:2
CAN_MAP_HOST_MEMORY:1
CLOCK_RATE:823500
COMPUTE_CAPABILITY_MAJOR:3
COMPUTE_CAPABILITY_MINOR:7
COMPUTE_MODE:DEFAULT
CONCURRENT_KERNELS:1
ECC_ENABLED:1
GLOBAL_L1_CACHE_SUPPORTED:1
GLOBAL_MEMORY_BUS_WIDTH:384
GPU_OVERLAP:1
INTEGRATED:0
KERNEL_EXEC_TIMEOUT:0
L2_CACHE_SIZE:1572864
LOCAL_L1_CACHE_SUPPORTED:1
MANAGED_MEMORY:1
MAXIMUM_SURFACE1D_LAYERED_LAYERS:2048
MAXIMUM_SURFACE1D_LAYERED_WIDTH:65536
MAXIMUM_SURFACE1D_WIDTH:65536
MAXIMUM_SURFACE2D_HEIGHT:32768
MAXIMUM_SURFACE2D_LAYERED_HEIGHT:32768
MAXIMUM_SURFACE2D_LAYERED_LAYERS:2048
MAXIMUM_SURFACE2D_LAYERED_WIDTH:65536
MAXIMUM_SURFACE2D_WIDTH:65536
MAXIMUM_SURFACE3D_DEPTH:2048
MAXIMUM_SURFACE3D_HEIGHT:32768
MAXIMUM_SURFACE3D_WIDTH:65536
MAXIMUM_SURFACECUBEMAP_LAYERED_LAYERS:2046
MAXIMUM_SURFACECUBEMAP_LAYERED_WIDTH:32768
MAXIMUM_SURFACECUBEMAP_WIDTH:32768
MAXIMUM_TEXTURE1D_LAYERED_LAYERS:2048
MAXIMUM_TEXTURE1D_LAYERED_WIDTH:16384
MAXIMUM_TEXTURE1D_LIN

In [ ]:
import numpy
a = numpy.random.randn(4,4)
a = a.astype(numpy.float32)
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

In [ ]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [ ]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)


print("Original Array: ", a)
print("Doubled Array: ", a_doubled)


Original Array:  [[-1.4181806  -1.0551018  -0.00773647  1.7427331 ]
 [ 0.34343663 -0.28739807  0.42590174  0.4297826 ]
 [-0.1457469   0.20598535  0.84063685  0.49176353]
 [-0.8859098  -0.43639514  1.8323126  -1.5600418 ]]
Doubled Array:  [[-2.8363612  -2.1102035  -0.01547294  3.4854662 ]
 [ 0.68687326 -0.57479614  0.8518035   0.8595652 ]
 [-0.2914938   0.4119707   1.6812737   0.98352706]
 [-1.7718196  -0.8727903   3.6646252  -3.1200836 ]]
